# 
# Utilização de Algorítmos de Machine Learning para Identificação de Empresas "de Fachada" em Operações de Importação
TCC PUC Minas - LUCIANA MACEDO RODRIGUES

### NOTEBOOK 1 - COLETA DOS DADOS

### 1 - Configurações iniciais

In [ ]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np

In [ ]:
# Versão do pandas em uso
pd.__version__

### 2 - Leitura das bases de dados e verificações preliminares

In [ ]:
dadosADU = pd.read_csv('arquivos/dadosADU.csv', sep = ';')

In [ ]:
dadosCNPJ = pd.read_csv('arquivos/dadosCNPJ.csv', sep = ';')

In [ ]:
dadosDIRF = pd.read_csv('arquivos/dadosDIRF.csv', sep = ';')

In [ ]:
dadosRB = pd.read_csv('arquivos/dadosRB.csv', sep = ';')

In [ ]:
# Linhas iniciais
dadosADU.head()

In [ ]:
# Tipos de Dados
dadosADU.dtypes

In [ ]:
# Quantidade de (linhas,colunas)
dadosADU.shape

In [ ]:
# Dados faltantes
dadosADU.isnull().sum()

In [ ]:
dadosCNPJ.head()

In [ ]:
dadosCNPJ.dtypes

In [ ]:
dadosCNPJ.shape

In [ ]:
dadosCNPJ.isnull().sum()

In [ ]:
dadosDIRF.head()

In [ ]:
dadosDIRF.dtypes

In [ ]:
dadosDIRF.shape

In [ ]:
dadosDIRF.isnull().sum()

In [ ]:
dadosRB.head()

In [ ]:
dadosRB.dtypes

In [ ]:
dadosRB.shape

In [ ]:
dadosRB.isnull().sum()

### 3 -  Junção das bases de dados

As bases de dados da DIRF e da RB não retornaram dados para todos os CNPJ consultados (evidenciado pela quantidade de linhas inferior à de dadosADU dessas bases), portanto existirão dados em branco na tabela resultante do join. 

In [ ]:
# Renomeando as colunas CNPJ de todos as bases de dados para padronizar
dadosADU.rename(columns={'ID_EMP':'ID'}, 
             inplace=True)
dadosCNPJ.rename(columns={'ID_EMP2':'ID'}, 
             inplace=True)
dadosDIRF.rename(columns={'ID_EMP3':'ID'}, 
             inplace=True)
dadosRB.rename(columns={'ID_EMP4':'ID'}, 
             inplace=True)

In [ ]:
# Verificando alterações em cada base de dados
dadosADU.dtypes

In [ ]:
dadosCNPJ.dtypes

In [ ]:
dadosDIRF.dtypes

In [ ]:
dadosRB.dtypes

In [ ]:
# O dataframe dadosCNPJ apresenta 1 linha a mais que o dadosADU. 
# Usando a função value_counts() confirmamos a suspeita de dados em duplicidade para um mesmo CNPJ
dadosCNPJ.ID.value_counts()

In [ ]:
# Localizando o índice das linhas de dadosCNPJ que contém o mesmo valor para avaliar a melhor opção para eliminar
np.where(dadosCNPJ["ID"] == 23809368)

In [ ]:
# São dados duplicados - dois registros em 2015 e 2017 da mesma empresa, eliminaremos o segundo na posição 12897
dadosCNPJ.drop(index=[12897],inplace=True)

# Verificando se a repetição foi eliminada
np.where(dadosCNPJ["ID"] == 23809368)

In [ ]:
# Juntando usando CNPJ como chave e left a partir de ADU>CNPJ>DIRF>RB (do maior volume de cnpj para o menor)
df_merged = pd.merge (dadosADU, dadosCNPJ, how = 'left', on = 'ID')
df_merged = pd.merge (df_merged, dadosDIRF, how = 'left', on = 'ID')
df_merged = pd.merge (df_merged, dadosRB, how = 'left', on = 'ID')

In [ ]:
# Obtendo todas as informações de df_merged
df_merged.info()

In [ ]:
df_merged.head()

### 4 - Formatação da base de dados df_merged
A base de dados resultante da junção das fontes de dados originais apresenta algumas colunas com dados desnecessários para o estudo, que eliminaremos a seguir.

As colunas 'SIT CAD EMP ADQUIR', 'MOTIVO SIT CAD EMP ADQUIR''Empresa - Sit. Cad. Atual (Nome)' e 'Empresa - Sit. Cad. Motivo Atual (Nome)' possuem dados semelhantes, porém extraídos em momentos diferentes, podendo apresentar divergências. Para verificar quais dados seriam mais interessantes para o estudo, optamos por analisá-los após a junção das bases.

Por fim, antes de passar à etapa de tratamento dos dados propriamente dita, reorganizaremos a ordem das colunas e as renomearemos.

In [ ]:
# Eliminando as colunas desnecessárias
df_merged.drop(columns=["ANO SIT CAD EMP ADQUIR", "ID", "EMP_CS_MIN"],inplace=True)

In [ ]:
df_merged.head()

In [ ]:
# Verificando se há divergência entre os dados de 'SIT CAD EMP ADQUIR' e 'Empresa - Sit. Cad. Atual (Nome)'
freq = df_merged.groupby(['SIT CAD EMP ADQUIR', 'MOTIVO SIT CAD EMP ADQUIR']).size()
freq2 = df_merged.groupby(['Empresa - Sit. Cad. Atual (Nome)', 'Empresa - Sit. Cad. Motivo Atual (Nome)']).size()
freq

In [ ]:
freq2

In [ ]:
# Há divergências, porém para fins do estudo, os motivos de interesse sofrem pouca alteração entre as duas bases.
# Como 'Empresa - Sit. Cad. Atual (Nome)' representa a situação cadastral ATUAL da empresa obtida 
# a partir do DATASET Cadastro CNPJ optamos por mantê-la, eliminando as colunas "SIT CAD EMP ADQUIR" e
# "MOTIVO SIT CAD EMP ADQUIR"
df_merged.drop(columns=["SIT CAD EMP ADQUIR", "MOTIVO SIT CAD EMP ADQUIR"],inplace=True)

In [ ]:
# Organizando as colunas no df_merged
df_merged = df_merged[['UF_EMP', 'EMP_PORTE', 'EMP_CS_MAX', 'IMP_VOL',
                       'IMP_VAL', 'IMP_PESO', 'IMP_CUSTO', 'EMP_EMPREG', 'EMP_REC', 'DT_ABERT_EMP', 
                       'Empresa - 6 Dia Sit. Cad. Atual (ID)', 'Empresa - Sit. Cad. Atual (Nome)', 
                       'Empresa - Sit. Cad. Motivo Atual (Nome)']]

In [ ]:
# Listando as colunas para renomear
list(df_merged.columns)

In [ ]:
# Renomeando as colunas
df_merged.rename(columns={'Empresa - 6 Dia Sit. Cad. Atual (ID)' : 'DT_SIT_CAD_EMP', 
                          'Empresa - Sit. Cad. Atual (Nome)': 'SIT_CAD_EMP', 
                          'Empresa - Sit. Cad. Motivo Atual (Nome)': 'MOT_SIT_CAD_EMP'}, 
             inplace=True)
list(df_merged.columns)

In [ ]:
# Criando csv de df_merged
df_merged.to_csv("arquivos/df_merged.csv", encoding = 'utf-8', index = False)

In [ ]:
# Verificando csv de df_merged
df_merged = pd.read_csv('arquivos/df_merged.csv', sep = ',')
df_merged.head()